In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Notes
- Simulation with using the final model and see if ur money goes up or down
- We need a benchmark to compare our model to.. someone elses prediction etc.
- good link for project: [click here](https://towardsdatascience.com/making-big-bucks-with-a-data-driven-sports-betting-strategy-6c21a6869171)
- ML, over/under, spread archive [click here](https://sportsbookreviewsonline.com/scoresoddsarchives/ncaabasketball/ncaabasketballoddsarchives.htm)
- Beating the bookies with their own numbers [click here](https://arxiv.org/abs/1710.02824)
    - Expected payout.. would be cool to come up with a probability of win or loss and then calculate expected payout
- NBA Sports Betting and PCA dimensionality reduction used [click here](https://medium.com/swlh/machine-learning-sports-betting-on-the-nba-season-before-the-bubble-bd6509be7e35)
- BetFair Api (backtesting and historical odds) costs money but there is a free version... look into this
    - [BetFair](https://developer.betfair.com/)
    - [sportsdata.io](https://sportsdata.io/)


## other notes
- Prediction winner, loser
- Predict over/under value
- Predict spread value

### models to use for binary classification (win or lose)
- Logistic Regression
- k-Nearest Neighbors
- Decision Trees
- Support Vector Machine
- Naive Bayes

In [2]:
df = pd.read_csv('data/bigten_data_real.csv')
df['Date'] = pd.to_datetime(df['Date'])
df_lines_1 = pd.read_excel('data/lines/ncaa_basketball_2016-17.xlsx')
df_lines_2 = pd.read_excel('data/lines/ncaa_basketball_2017-18.xlsx')
df_lines_3 = pd.read_excel('data/lines/ncaa_basketball_2018-19.xlsx')
df_lines_4 = pd.read_excel('data/lines/ncaa_basketball_2019-20.xlsx')
df_lines_5 = pd.read_excel('data/lines/ncaa_basketball_2020-21.xlsx')

In [3]:
def getDate(date_int, start_year):
    if date_int >= 1000:
        year = start_year
        return datetime.strptime(str(date_int)+year, '%m%d%Y')
    else:
        year = str(int(start_year) + 1)
        return datetime.strptime('0'+str(date_int)+year, '%m%d%Y')

In [4]:
df_lines_1['Date'] = df_lines_1.apply(lambda row: getDate(row['Date'], '2016'), axis=1)
df_lines_2['Date'] = df_lines_2.apply(lambda row: getDate(row['Date'], '2017'), axis=1)
df_lines_3['Date'] = df_lines_3.apply(lambda row: getDate(row['Date'], '2018'), axis=1)
df_lines_4['Date'] = df_lines_4.apply(lambda row: getDate(row['Date'], '2019'), axis=1)
df_lines_5['Date'] = df_lines_5.apply(lambda row: getDate(row['Date'], '2020'), axis=1)

In [5]:
df_lines = pd.concat([df_lines_1, df_lines_2, df_lines_3, df_lines_4, df_lines_5])

In [6]:
df_lines = df_lines[df_lines.columns[:10]]

In [7]:
df_lines.head()

,Date,Rot,VH,Team,1st,2nd,Final,Open,Close,ML
0,2016-11-11,717,V,Evansville,27,20,47,135.5,141,2500
1,2016-11-11,718,H,Louisville,43,35,78,17,22.5,-5000
2,2016-11-11,719,V,Georgia,30,34,64,132.5,139.5,180
3,2016-11-11,720,H,Clemson,40,34,74,6,5,-210
4,2016-11-11,721,V,GeorgiaSouthern,36,43,79,146,158,800


In [8]:
team_map = {'Penn State': 'PennState', 'Ohio State': 'OhioState',  'Michigan State': 'MichiganState'}
def get_MLs(row):
    home = team_map[row['Home']] if row['Home'] in list(team_map.keys()) else row['Home']
    date = row['Date']
        
    home_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==home) & (df_lines['VH']=='H')]
    
    if home_lines.empty: 
        home_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==home) & (df_lines['VH']=='N')]
        if home_lines.empty:
            home_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==home+'U') & (df_lines['VH']=='H')]
            if home_lines.empty:
                home_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==home+'U') & (df_lines['VH']=='N')]
                if home_lines.empty:
                    raise Exception(f'home_lines empty... {home}, {date}')
    home_ml = home_lines['ML'].values[0]
    if type(home_ml) == str and home_ml == 'NL':
        home_ml = np.nan
    
    away = team_map[row['Away']] if row['Away'] in list(team_map.keys()) else row['Away']
    date = row['Date']
        
    away_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==away) & (df_lines['VH']=='V')]
    
    if away_lines.empty: 
        away_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==away) & (df_lines['VH']=='N')]
        if away_lines.empty:
            away_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==away+'U') & (df_lines['VH']=='V')]
            if away_lines.empty:
                away_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==away+'U') & (df_lines['VH']=='N')]
                if home_lines.empty:
                    raise Exception(f'home_lines empty... {home}, {date}')
                    
    away_ml = away_lines['ML'].values[0]
    if type(away_ml) == str and away_ml == 'NL':
        away_ml = np.nan
    
    return home_ml, away_ml


def get_spread_tot(row):
    home = team_map[row['Home']] if row['Home'] in list(team_map.keys()) else row['Home']
    away = team_map[row['Away']] if row['Away'] in list(team_map.keys()) else row['Away']
    date = row['Date']
        
    home_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==home) & (df_lines['VH']=='H')]
    
    if home_lines.empty: 
        home_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==home) & (df_lines['VH']=='N')]
        if home_lines.empty:
            home_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==home+'U') & (df_lines['VH']=='H')]
            if home_lines.empty:
                home_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==home+'U') & (df_lines['VH']=='N')]
                if home_lines.empty:
                    raise Exception(f'home_lines empty... {home}, {date}')
                    
    away_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==away) & (df_lines['VH']=='V')]
    
    if away_lines.empty: 
        away_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==away) & (df_lines['VH']=='N')]
        if away_lines.empty:
            away_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==away+'U') & (df_lines['VH']=='V')]
            if away_lines.empty:
                away_lines = df_lines[(df_lines['Date']==date) & (df_lines['Team']==away+'U') & (df_lines['VH']=='N')]
                if home_lines.empty:
                    raise Exception(f'home_lines empty... {home}, {date}')

    home_open = home_lines['Open'].values[0]
    home_close = home_lines['Close'].values[0]
    away_open = away_lines['Open'].values[0]
    away_close = away_lines['Close'].values[0]
    
    if home_open == 'pk' and home_close == 'pk':
        if away_open == 'pk' and away_close == 'pk':
            open_spread, close_spread, open_tot, close_tot = np.nan, np.nan, np.nan, np.nan
        if away_open == 'pk':
            if abs(int(away_close)) > 40:
                open_spread, close_spread = np.nan, np.nan
                open_tot, close_tot = np.nan, int(away_close)
            else:
                open_spread, close_spread = np.nan, int(away_close)
                open_tot, close_tot = np.nan, np.nan
        if away_close == 'pk':
            if abs(int(away_open)) > 40:
                open_spread, close_spread = np.nan, np.nan
                open_tot, close_tot = int(away_open), np.nan
            else:
                open_spread, close_spread = int(away_open), np.nan
                open_tot, close_tot = np.nan, np.nan
                
        if away_open != 'pk' and away_close != 'pk':
            if abs(int(away_open)) > 40:
                open_tot = int(away_open)
                open_spread = np.nan
            else:
                open_spread = int(away_open)
                open_tot = np.nan
            
            if abs(int(away_close)) > 40:
                close_tot = int(away_close)
                close_spread = np.nan
            else:
                close_spread = int(away_close)
                close_tot = np.nan
                
        return open_spread, close_spread, open_tot, close_tot
                
    if away_open == 'pk' and away_close == 'pk':
        if home_open == 'pk' and home_close == 'pk':
            open_spread, close_spread, open_tot, close_tot = np.nan, np.nan, np.nan, np.nan
        if home_open == 'pk':
            if abs(int(home_close)) > 40:
                open_spread, close_spread = np.nan, np.nan
                open_tot, close_tot = np.nan, int(home_close)
            else:
                open_spread, close_spread = np.nan, int(home_close)
                open_tot, close_tot = np.nan, np.nan
        if home_close == 'pk':
            if abs(int(home_open)) > 40:
                open_spread, close_spread = np.nan, np.nan
                open_tot, close_tot = int(home_open), np.nan
            else:
                open_spread, close_spread = int(home_open), np.nan
                open_tot, close_tot = np.nan, np.nan
                
        if home_open != 'pk' and home_close != 'pk':
            if abs(int(home_open)) > 40:
                open_tot = int(home_open)
                open_spread = np.nan
            else:
                open_spread = int(home_open)
                open_tot = np.nan
            
            if abs(int(away_close)) > 40:
                close_tot = int(home_open)
                close_spread = np.nan
            else:
                close_spread = int(home_open)
                close_tot = np.nan
                
        return open_spread, close_spread, open_tot, close_tot
                
    
    if home_open == 'pk':
        if abs(int(home_close)) > 40:
            
            close_tot = int(home_close)

            if away_open == 'pk':
                open_tot, open_spread = np.nan, np.nan
            else:
                if abs(int(away_open)) > 40:
                    open_tot = int(away_open)
                    open_spread = np.nan
                else:
                    open_spread = int(away_open)
                    open_tot = np.nan
                    
            if away_close == 'pk':
                close_spread = np.nan
            else:
                close_spread = int(away_close)
            
        else:
            
            close_spread = int(home_close)

            if away_open == 'pk':
                open_tot, open_spread = np.nan, np.nan
            else:
                if abs(int(away_open)) > 40:
                    open_tot = int(away_open)
                    open_spread = np.nan
                else:
                    open_spread = int(away_open)
                    open_tot = np.nan  
                    
            if away_close == 'pk':
                close_tot = np.nan
            else:
                close_tot = int(away_close)
                    
        return open_spread, close_spread, open_tot, close_tot
                    
                    
    if home_close == 'pk':
        if abs(int(home_open)) > 40:
            
            open_tot = int(home_open)

            if away_close == 'pk':
                close_tot, close_spread = np.nan, np.nan
            else:
                if abs(int(away_close)) > 40:
                    close_tot = int(away_close)
                    close_spread = np.nan
                else:
                    close_spread = int(away_close)
                    close_tot = np.nan
                    
            if away_open == 'pk':
                open_spread = np.nan
            else:
                open_spread = int(away_open)
            
        else:
            
            open_spread = int(home_open)

            if away_close == 'pk':
                close_tot, close_tot = np.nan, np.nan
            else:
                if abs(int(away_close)) > 40:
                    close_tot = int(away_close)
                    close_spread = np.nan
                else:
                    close_spread = int(away_close)
                    close_tot = np.nan  
                    
            if away_open == 'pk':
                open_tot = np.nan
            else:
                open_tot = int(away_open)
                    
        return open_spread, close_spread, open_tot, close_tot  
    
    
    
    if away_open == 'pk':
        if abs(int(away_close)) > 40:
            
            close_tot = int(away_close)

            if home_open == 'pk':
                open_tot, open_spread = np.nan, np.nan
            else:
                if abs(int(home_open)) > 40:
                    open_tot = int(home_open)
                    open_spread = np.nan
                else:
                    open_spread = int(home_open)
                    open_tot = np.nan
                    
            if home_close == 'pk':
                close_spread = np.nan
            else:
                close_spread = int(home_close)
            
        else:
            
            close_spread = int(away_close)

            if home_open == 'pk':
                open_tot, open_spread = np.nan, np.nan
            else:
                if abs(int(home_open)) > 40:
                    open_tot = int(home_open)
                    open_spread = np.nan
                else:
                    open_spread = int(home_open)
                    open_tot = np.nan  
                    
            if home_close == 'pk':
                close_tot = np.nan
            else:
                close_tot = int(home_close)
                    
        return open_spread, close_spread, open_tot, close_tot
    
    
    if away_close == 'pk':
        if abs(int(away_open)) > 40:
            
            open_tot = int(away_open)

            if home_close == 'pk':
                close_tot, close_spread = np.nan, np.nan
            else:
                if abs(int(home_close)) > 40:
                    close_tot = int(home_close)
                    close_spread = np.nan
                else:
                    close_spread = int(home_close)
                    close_tot = np.nan
                    
            if home_open == 'pk':
                open_spread = np.nan
            else:
                open_spread = int(home_open)
            
        else:
            
            open_spread = int(away_open)

            if home_close == 'pk':
                close_tot, close_tot = np.nan, np.nan
            else:
                if abs(int(home_close)) > 40:
                    close_tot = int(home_close)
                    close_spread = np.nan
                else:
                    close_spread = int(home_close)
                    close_tot = np.nan  
                    
            if home_open == 'pk':
                open_tot = np.nan
            else:
                open_tot = int(home_open)
                    
        return open_spread, close_spread, open_tot, close_tot  
    
    # at this point, none are = 'pk'
    
    home_open = int(home_open)
    home_close = int(home_close)
    away_open = int(away_open)
    away_close = int(away_close)
    
        
    if abs(home_open) > abs(away_open):
        open_spread = away_open
        open_tot = home_open
    else:
        open_spread = home_open
        open_tot = away_open
        
    if abs(home_close) > abs(away_close):
        close_spread = away_close
        close_tot = home_close
    else:
        close_spread = home_close
        close_tot = away_close
    
    
    return open_spread, close_spread, open_tot, close_tot

In [9]:
df = pd.read_csv('data/bigten_data_real.csv')

df['open_spread'], df['close_spread'], df['open_tot'], df['close_tot'] = zip(*df.apply(lambda row: get_spread_tot(row), axis=1))
df['Home_ML'], df['Away_ML'] = zip(*df.apply(lambda row: get_MLs(row), axis=1))

In [10]:
df = df.sort_values('Date', ascending=True)
df = df.reset_index(drop=True)

In [11]:
df = df.reset_index()

In [12]:
df = df.rename(columns={'index': 'game_id'})

In [13]:
df['Tot'] = df.Home_PTS + df.Away_PTS
df['Spread'] = abs(df.Home_PTS - df.Away_PTS)

In [18]:
df

,game_id,Date,Home,Home_PTS,Away,Away_PTS,Playoffs,Home_opponent-effective-possession-ratio,Home_opponent-effective-possession-ratio_last1,Home_opponent-effective-possession-ratio_last3,...,OT,winner,open_spread,close_spread,open_tot,close_tot,Home_ML,Away_ML,Tot,Spread
0,0,2016-12-27,Maryland,84,Illinois,59,0,0.949,0.873,0.916,...,0,1,4.0,3.0,138,139,-150.0,130.0,143,25
1,1,2016-12-27,Minnesota,74,Michigan State,75,0,0.932,0.807,0.864,...,1,0,5.0,5.0,139,138,-260.0,210.0,149,1
2,2,2016-12-27,Penn State,77,Northwestern,87,0,0.914,0.847,0.921,...,0,0,1.0,3.0,142,139,145.0,-165.0,164,10
3,3,2016-12-27,Wisconsin,72,Rutgers,52,0,0.886,0.829,0.869,...,0,1,18.0,17.0,128,128,NaN,NaN,124,20
4,4,2016-12-28,Indiana,83,Nebraska,87,0,0.962,0.861,0.940,...,0,0,15.0,13.0,142,144,-1300.0,850.0,170,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,716,2021-03-12,Iowa,62,Wisconsin,57,1,0.984,0.926,0.870,...,0,1,4.0,5.0,146,143,-200.0,175.0,119,5
717,717,2021-03-12,Purdue,78,Ohio State,87,1,0.940,0.879,0.945,...,1,0,1.0,1.0,142,140,100.0,-120.0,165,9
718,718,2021-03-13,Illinois,82,Iowa,71,1,0.938,0.879,0.929,...,0,1,2.0,3.0,157,154,-155.0,135.0,153,11
719,719,2021-03-13,Michigan,67,Ohio State,68,1,0.958,0.890,0.938,...,0,0,6.0,4.0,146,142,-230.0,195.0,135,1


In [15]:
df.to_csv('data/bigten_data_joined.csv', index=False)